<a href="https://colab.research.google.com/github/saurabhburewar/ML_RL-test-case-prioritization/blob/main/TestCasePrioritization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!pip install stable-baselines[mpi]==2.10.2

     |████████████████████████████████| 240 kB 4.9 MB/s 
     |████████████████████████████████| 2.5 MB 54.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185309 sha256=75e8877dd94fe8e89abd3f1222027583448c1f215cfffda5ae92163ddb2d6f54
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


In [2]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 1.1 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 3.2 MB 47.1 MB/s 
     |████████████████████████████████| 367 kB 57.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you

In [3]:
!pip install wrapt-timeout-decorator

##Define each Cycle

In [4]:
import numpy as np
from sklearn import preprocessing
import random
import copy

from typing import Any, Union
import gym
from gym import spaces


from stable_baselines.common.vec_env import DummyVecEnv
from wrapt_timeout_decorator import *

import pandas as pd
import os
from pprint import pprint


winSize = 5
numCycles = 350
steps = 10000
data = "https://raw.githubusercontent.com/saurabhburewar/SDE_Test-case-prioritization/main/data/tc_data_paintcontrol.csv"

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [5]:
class CycleObj:
    testCase = {}
    cId = 0

    def __init__(self, cId: int):
        self.cId = cId
        self.testCases = []
        self.failedCases = 0
        self.totalCases = 0


    def addTest(self, tId, tName, avgTime: int, lastTime: int, fail: int,failHistory: list, execHistory: list):
        """
        Adds test case to the test case list

        Args:
            tId (_type_): _description_
            tName (_type_): _description_
            avgTime (int): _description_
            lastTime (int): _description_
            fail (int): _description_
            failHistory (list): _description_
            execHistory (list): _description_
        """
        testCase = {
            'tId': tId,
            'tName': tName,
            'avgTime': avgTime,
            'lastTime': lastTime,
            'fail': fail
        }

        if not failHistory:
            failHistory = []

        testCase['failHistory'] = failHistory
        testCase['age'] = len(failHistory)

        # if failHistory:
        #     testCase['failHistory'] = failHistory
        #     testCase['age'] = len(failHistory)
        # else:
        #     testCase['failHistory'] = []
        #     testCase['age'] = 0
        
        if not execHistory:
            execHistory = None
        else:
            testCase['execHistory'] = execHistory
        
        self.testCases.append(testCase)
        self.totalCases += 1
        self.failedCases += fail


    def export(self, testCase: dict, winSize=4):
        """Returns the test case vector containing details about test case failure history and its average complexity/other matrix and time group and duration group and avearge execution time and age

        Args:
            testCase (dict): _description_
            option (str): _description_
            winSize (int, optional): _description_. Defaults to 4.

        Returns:
            _type_: _description_
        """

        testCaseVector = np.zeros((winSize + 4))
        i1 = 0
        for j in range(0, min(len(testCase['failHistory']), winSize)):
            testCaseVector[j] = testCase['failHistory'][j]
            i1 = i1 + 1

        for j in range(len(testCase['failHistory']), winSize):
            testCaseVector[j] = -1
            i1 = i1 + 1


        testCaseVector[i1] = testCase['avgTime']
        testCaseVector[i1+1] = testCase['age']

        return testCaseVector


In [6]:
def getSortedByKey(toSort, sortKey, toRev):
    sortedOrder = sorted(toSort, key=lambda x: x[sortKey], reverse=toRev)
    return sortedOrder

In [7]:
class Calculation:
    def __init__(self):
        pass

    
    @staticmethod
    def calAPFDOpt(totalCases, failedCases, testCases):
        """Calculates optimal APFD for the sorted test cases

        Returns:
             _type_: _description_
        """
        optOrder = getSortedByKey(testCases, 'fail', True)
        ranksSum = 0
        i = 1
        fault = 0
        for testCase in optOrder:
            ranksSum = ranksSum + testCase['fail'] * i
            i = i+1
        if totalCases > 0 and failedCases > 0:
            fault = 1 - (ranksSum / (totalCases * failedCases)) + (1 / (2 * totalCases))
        return fault


    @staticmethod
    def calAPFDRandom(totalCases, failedCases, testCases):
        """Calculates APFD score for random not particularly sorted order of test cases

        Returns:
            _type_: _description_
        """
        ranOrder = random.sample(testCases, len(testCases))

        ranFault = 0
        ranksSum = 0
        for i in range(0, len(ranOrder)):
            ranksSum = ranksSum + ranOrder[i]['fail'] * (i + 1)

        if totalCases > 0 and failedCases > 0:
            ranFault = 1 - (ranksSum / (totalCases * failedCases)) + (1 / (2 * totalCases))
        return ranFault

    @staticmethod
    def calAPFD(testCaseProb, threshold, totalCases, failedCases, testCases):
        """Calculate APFD score for test case list sorted by probability

        Args:
            testCaseProb (list): _description_
            threshold (float): _description_

        Returns:
            _type_: _description_
        """
        ranksSum = 0
        fault = 0
        i = 1
        testCaseProb = getSortedByKey(testCaseProb, 'prob', False)
        for prob in testCaseProb:
            ranksSum = ranksSum + testCases[prob['index']]['fail'] * i
            i = i+1

        if totalCases > 0 and failedCases > 0:
            fault = 1 - (ranksSum / (totalCases * failedCases)) + (1 / (2 * totalCases))
        return fault

    @staticmethod
    def getOptOrder(totalCases, failedCases, testCases):
        """Returns optimal order of the test cases
            first test cases are sorted by fail (pass / fail)    
                    with fail as priority 
            Among the failed test cases, test case 
                    with lease execution time are given priority and 
            same among the passed test cases

        Returns:
            _type_: _description_
        """
        optFailOrder = copy.deepcopy(getSortedByKey(testCases, 'fail', True))
        failOrder = []
        passOrder = []
        failOrder.extend(getSortedByKey(optFailOrder[0:failedCases], 'lastTime', False))
        passOrder.extend(getSortedByKey(optFailOrder[failedCases:], 'lastTime', False))
        failOrder.extend(passOrder)
        return failOrder

    @staticmethod
    def addTestToCycle(cobj, id, name, duration1, duration2, verdict, lastResults, execHis):
      cobj.addTest(id, name, duration1, duration2, verdict, lastResults, execHis)
      return cobj


## TP Point Wise Env

In [8]:
class CIEnv(gym.Env):
    def __init__(self, cObj: CycleObj, winSize):
        super(CIEnv, self).__init__()
        self.winSize = winSize
        self.cObj = cObj
        self.currI = 0
        self.optOrder= Calculation.getOptOrder(cObj.totalCases, cObj.failedCases, cObj.testCases)
        self.testCaseSize = self.winSize + 4
        self.currObs = np.zeros((1, self.testCaseSize))
        self.initObs = self.cObj.export(self.cObj.testCases[self.currI],self.winSize)
        self.currObs = self.initObs.copy()

        self.action_space = spaces.Box(low=0, high=1, shape=(1, ))
        self.observation_space = spaces.Box(low=0, high=1,
                                            shape=(1, self.testCaseSize))


    def reset(self):
        self.currI = 0
        self.currObs = self.cObj.export(self.cObj.testCases[self.currI],self.winSize)
        return self.getObs()

    def getObs(self):
      return self.currObs
    
    def normalize(self, rank):
      rankOpt = rank/self.cObj.totalCases
      temp = rankOpt - 1
      rankOpt += (2/temp) * temp - 1
      return rankOpt
    
    def rankBasedReward(self, prob, rank):
        reward = (prob-rank)**2
        reward = 1 - reward
        return reward


    ## the reward function must be called before updating the observation
    def calReward(self, prob):
        prob = prob[0]
        rankOpt= self.optOrder.index(self.cObj.testCases[self.currI])
        rankOpt= self.normalize(rankOpt)
        return self.rankBasedReward(prob, rankOpt)

    def step(self, prob):
        done = False
        reward = self.calReward(prob)
        prob = {'index': self.currI, 'prob': prob}
        self.currI = self.currI + 1
        if self.currI < self.cObj.totalCases:
            self.currObs = self.cObj.export(self.cObj.testCases[self.currI],self.winSize)
        else:
            done = True
            self.currObs = np.zeros(self.testCaseSize)
        return self.currObs, reward, done, {}


## TP Point Wise Agent

In [9]:
class TestAgent:

    def train(self, env: CIEnv, steps: int, path_to_save_agent, base=None, algo="PPO2"):
        env.reset()
        if not base:
            if algo.upper() == "PPO2":
              from stable_baselines.common.policies import MlpPolicy
              from stable_baselines import PPO2
              base = PPO2(MlpPolicy, env, gamma=0.90, learning_rate=0.0005, n_steps=steps)
              print("Agent created")

            elif algo.upper() == "A2C":
              from stable_baselines.common.policies import MlpPolicy
              from stable_baselines.a2c import A2C
              env = DummyVecEnv([lambda: env])
              base = A2C(MlpPolicy, env, gamma=0.90, learning_rate=0.0005,
                          n_cpu_tf_sess=None, verbose=0,
                          tensorboard_log=None, _init_setup_model=True, policy_kwargs=None,
                          full_tensorboard_log=False, seed=None)
              print("Agent created")
              
            elif algo.upper() == "TRPO":
              from stable_baselines.common.policies import MlpPolicy
              from stable_baselines.trpo_mpi import TRPO
              env = DummyVecEnv([lambda: env])
              base = TRPO(MlpPolicy, env, verbose=0)
              print("Agent created")

            elif algo.upper() == "TD3":
              from stable_baselines.td3.policies import MlpPolicy
              from stable_baselines import TD3
              env = DummyVecEnv([lambda: env])
              base = TD3(MlpPolicy, env, verbose=0)
              print("Agent created")

            elif algo.upper() == "ACKTR":
              from stable_baselines.common.policies import MlpPolicy
              from stable_baselines.acktr import ACKTR
              env = DummyVecEnv([lambda: env])
              base = ACKTR(MlpPolicy, env, verbose=0)
              print("Agent created")

        else:
            env = DummyVecEnv([lambda: env])
            base.set_env(env)
        
        base = base.learn(total_timesteps=steps, reset_num_timesteps=False)
        base.save(path_to_save_agent)
        return base


    def test(self, env: CIEnv, model_path: str, model, algo="PPO2"):
        agent_actions = []
        print("Evaluation of an agent from " + model_path)
        if not model:
            if algo.upper() == "PPO2":
              from stable_baselines import PPO2
              model = PPO2.load(model_path)
              print("Agent loaded")

            elif algo.upper() == "A2C":
              from stable_baselines.a2c import A2C
              model = A2C.load(model_path)
              print("Agent loaded")

            elif algo.upper() == "TRPO":
              from stable_baselines.trpo_mpi import TRPO
              model = TRPO.load(model_path)
              print("Agent loaded")

            elif algo.upper() == "TD3":
              from stable_baselines.td3.policies import MlpPolicy
              from stable_baselines import TD3
              model = TD3(MlpPolicy, env, verbose=0)
              print("Agent loaded")

            elif algo.upper() == "ACKTR":
              from stable_baselines.acktr import ACKTR
              model = ACKTR.load(model_path)
              print("Agent loaded")

        else:
            testCases = env.cObj.testCases
            env = DummyVecEnv([lambda: env])
            model.set_env(env)
            obs = env.reset()
            testCasesProb = []
            for i in range(0, len(testCases)):
                action, _states = model.predict(obs, deterministic=False)

                obs, rewards, done, info = env.step(action)
                testCasesProb.append({'index': i, 'prob': action})
        return testCasesProb

##TP Point Wise CMD file

In [10]:
def headDivider(header:str):
    print()
    print("---------------------------------------------------------------------------------------------")
    print("\t\t" + header)
    print("---------------------------------------------------------------------------------------------")
    print()


def divider():
    print()
    print("---------------------------------------------------------------------------------------------")
    print()

In [11]:
class LogFrame:
    def __init__(self):
        self.logger = pd.DataFrame(columns=["cId","totalCases","failedCases","apfd","ranFault","apfdOpt"])

    def append(self, data):
        self.logger = self.logger.append(data, ignore_index=True)

    def saveToFile(self, algo):
        self.logger.to_csv("./evaluation_logs_{}.csv".format(algo))
    
    def mean(self, key):
        return self.logger[key].mean()

    def getLastRow(self, key):
        return self.logger.iloc[-1][key]

In [12]:
def run_model(algo, dfoutavg):
    headDivider("Running {} algorithm for Test Case Prioritization".format(algo.upper()))
    cycleOne = True
    agent = TestAgent()

    dfout = LogFrame()
    
    

    for i in range(1, 1 + numCycles - 1):
      if cycleObjs[i].totalCases <= 1 or cycleObjs[i].failedCases <= 1:
          continue
      ncTotalCases = cycleObjs[i+1].totalCases
      print("Training the agent by replaying cycle " + str(i))
      env = CIEnv(cycleObjs[i], winSize)
      if cycleOne:
          model = agent.train(env=env, steps=steps, path_to_save_agent="_pointwise_" + "_" + str(i), algo=algo)
          cycleOne = False
      else:
          model = agent.train(env=env, steps=steps, path_to_save_agent="_pointwise_" + "_" + str(i), base=model)

      print("Training agent by replaying of cycle " + str(i) + " is finished")

      # test the model on the next cycle
      if ncTotalCases <= 1:
          continue
      env_test = CIEnv(cycleObjs[i + 1], winSize)

      try:
          testCaseProb = agent.test(env=env_test, model=model,
                                              model_path="DQN_prioritization_" + "_" + str(i), algo=algo)
      except TimeoutError:
          print("Testing agent on cycle " + str(i + 1) + " timed out")
          continue
      print()

      ncTotalCases = cycleObjs[i+1].totalCases
      ncFailedCases = cycleObjs[i+1].failedCases
      ncTestCases = cycleObjs[i+1].testCases
      # apfd = cycleObjs[i + 1].calAPFD(testCaseProb, .80)
      apfd = Calculation.calAPFD(testCaseProb, 0.80, ncTotalCases, ncFailedCases, ncTestCases)
      apfdOpt = Calculation.calAPFDOpt(ncTotalCases, ncFailedCases, ncTestCases)

      apfdRan = Calculation.calAPFDRandom(ncTotalCases, ncFailedCases, ncTestCases)
      # failed = cycleObjs[i + 1].failedCases
      # total = cycleObjs[i + 1].totalCases

      # pResult(i+1, apfd, apfdOpt, apfdRan, failed, total)

      dfoutItem = {
          "cId": i + 1,
          "totalCases": ncTotalCases,
          "failedCases": ncFailedCases,
          "apfd": apfd,
          "ranFault": apfdRan,
          "apfdOpt": apfdOpt
      }
      pprint(dfoutItem)
      divider()
      dfout.append(dfoutItem)

    print()

    dfoutavgitem = {
        "algo": algo.upper(),
        "cId": dfout.getLastRow('cId'),
        "totalCases": dfout.getLastRow('totalCases'),
        "failedCases": dfout.getLastRow('failedCases'),
        "apfd": dfout.getLastRow('apfd'),
        "ranFault": dfout.getLastRow('ranFault'),
        "apfdOpt": dfout.getLastRow('apfdOpt'),
        "avgapfd": dfout.mean("apfd"),
        "avgranFault": dfout.mean("ranFault"),
        "avgapfdOpt": dfout.mean("apfdOpt")
    }
    dfoutavg = dfoutavg.append(dfoutavgitem, ignore_index=True)
    dfout.saveToFile(algo)
    dfoutavg.to_csv("./evaluation_logs_avg.csv")

In [13]:
# open data
df = pd.read_csv(data, error_bad_lines=False, sep=";")
print("Total Test Cases : ", df.shape[0])
divider()

dfoutavg = pd.DataFrame(columns=["algo","cId","totalCases","failedCases","apfd","ranFault","apfdOpt","avgapfd","avgranFault","avgapfdOpt"])

# process last result
LastRes = []
for i in range(df.shape[0]):
    LastRess = df["LastResults"][i]
    templ = LastRess.strip('][').split(', ')
    LastRes.append(templ)

df["LastResults"] = LastRes.copy()

# find start and end cycle id
start = 9999
for i in range(df.shape[0]):
    if(start > df["Cycle"][i]):
        start = int(df["Cycle"][i])

end = -1
for i in range(df.shape[0]):
    if(end < df["Cycle"][i]):
        end = int(df["Cycle"][i])

cycleObjs = []

# process all cycles and save them in a list of CycleObjs
for i in range(start, end + 1):
    cycleObj = CycleObj(i)
    cycleData = df.loc[df['Cycle'] == i]
    for index, test in cycleData.iterrows():
        cycleObj = Calculation.addTestToCycle(cycleObj, test["Id"], test["Name"], test["Duration"], test["Duration"], test["Verdict"], test["LastResults"], None)
    cycleObjs.append(cycleObj)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Total Test Cases :  25594

---------------------------------------------------------------------------------------------



In [14]:
# training using n cycle staring from start cycle
run_model(algo = "ppo2", dfoutavg=dfoutavg)


---------------------------------------------------------------------------------------------
		Running PPO2 algorithm for Test Case Prioritization
---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 9
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Agent created
Training agent by replaying of cycle 9 is finished
Evaluation of an agent from DQN_prioritization__9

{'apfd': 0.6186440677966101,
 'apfdOpt': 0.9661016949152542,
 'cId': 10,
 'failedCases': 4,
 'ranFault': 0.4661016949152542,
 'totalCases': 59}

---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 10
Training agent by replaying of cycle 10 is finished
Evaluation of an agent from DQN_prioritization__10

{'apfd': 0.6611111111111111,
 'apf

In [15]:
run_model(algo = "a2c", dfoutavg=dfoutavg)


---------------------------------------------------------------------------------------------
		Running A2C algorithm for Test Case Prioritization
---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 9
Agent created
Training agent by replaying of cycle 9 is finished
Evaluation of an agent from DQN_prioritization__9

{'apfd': 0.6398305084745762,
 'apfdOpt': 0.9661016949152542,
 'cId': 10,
 'failedCases': 4,
 'ranFault': 0.597457627118644,
 'totalCases': 59}

---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 10
Training agent by replaying of cycle 10 is finished
Evaluation of an agent from DQN_prioritization__10

{'apfd': 0.7685185185185185,
 'apfdOpt': 0.95,
 'cId': 11,
 'failedCases': 9,
 'ranFault': 0.5895061728395061,
 'totalCases': 90}

--------------------------------------------------------------------------------------

In [16]:
run_model(algo = "trpo", dfoutavg=dfoutavg)


---------------------------------------------------------------------------------------------
		Running TRPO algorithm for Test Case Prioritization
---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 9
Agent created
Training agent by replaying of cycle 9 is finished
Evaluation of an agent from DQN_prioritization__9

{'apfd': 0.6398305084745762,
 'apfdOpt': 0.9661016949152542,
 'cId': 10,
 'failedCases': 4,
 'ranFault': 0.21610169491525424,
 'totalCases': 59}

---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 10
Training agent by replaying of cycle 10 is finished
Evaluation of an agent from DQN_prioritization__10

{'apfd': 0.6858024691358025,
 'apfdOpt': 0.95,
 'cId': 11,
 'failedCases': 9,
 'ranFault': 0.5475308641975308,
 'totalCases': 90}

-----------------------------------------------------------------------------------

In [ ]:
run_model(algo = "td3", dfoutavg=dfoutavg)


---------------------------------------------------------------------------------------------
		Running TD3 algorithm for Test Case Prioritization
---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 9
Instructions for updating:
Use keras.layers.dense instead.
Agent created
Training agent by replaying of cycle 9 is finished
Evaluation of an agent from DQN_prioritization__9

{'apfd': 0.6398305084745762,
 'apfdOpt': 0.9661016949152542,
 'cId': 10,
 'failedCases': 4,
 'ranFault': 0.559322033898305,
 'totalCases': 59}

---------------------------------------------------------------------------------------------

Training the agent by replaying cycle 10
Training agent by replaying of cycle 10 is finished
Evaluation of an agent from DQN_prioritization__10

{'apfd': 0.7685185185185185,
 'apfdOpt': 0.95,
 'cId': 11,
 'failedCases': 9,
 'ranFault': 0.47592592592592586,
 'totalCases': 90}

--------------------------

In [ ]:
run_model(algo = "acktr", dfoutavg=dfoutavg)